# Importance Weights analysis

### 1 - Import Data

In [2]:
import os
import numpy as np
import pandas as pd
import json

In [3]:
os.chdir('..')

In [4]:
sim_path = '../results/synth_classification/27-04-2023-15:37'

In [5]:
def get_immediate_subdirectories(a_dir):
    return [name for name in os.listdir(a_dir)
            if os.path.isdir(os.path.join(a_dir, name))]

runs_dirs = get_immediate_subdirectories(sim_path)

In [9]:
true_weights_list = []
pre_AL_weights_list = []

uncertainty_spreading_weight_list = []
first_K_sampling_weight_list = []
random_sampling_weight_list = []

for run in runs_dirs:
    with open(sim_path + '/'+ run +'/true_weights.json', 'r') as f:
        true_weights_list.append(json.load(f))

    with open(sim_path + '/'+ run +'/pre_AL_weights.json', 'r') as f:
        pre_AL_weights_list.append(json.load(f))

    with open(sim_path + '/'+ run +'/UncertaintySpreading.json', 'r') as f:
        uncertainty_spreading_weight_list.append(json.load(f))

    with open(sim_path + '/'+ run +'/FirstKSampling.json', 'r') as f:
        first_K_sampling_weight_list.append(json.load(f))

    with open(sim_path + '/'+ run +'/RandomSampling.json', 'r') as f:
        random_sampling_weight_list.append(json.load(f))